In [21]:
import pandas as pd
import folium
import json
import requests
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv('seasonal_summary_final_fahrenheit.csv') #input data
df_wind = pd.read_csv('seasonal_summary_wind_full.csv')

In [9]:
# climate scores for each county -- Now includes w/ WIND)
def calculate_county_scores(df, df_wind):
    scores = []
    df = df[df['county_name'] != 'Sutter']
    
    for county in df['county_name'].unique():
        county_data = df[df['county_name'] == county]
        
        # temp comfort score
        avg_temp = county_data['TAVG'].mean()
        if 60 <= avg_temp <= 70:
            temp_comfort_score = 100
        elif avg_temp < 60:
            temp_comfort_score = max(0, 100 - (60 - avg_temp) * 4)
        else:
            temp_comfort_score = max(0, 100 - (avg_temp - 70) * 4)
        
        # temp stability score
        temp_range = county_data['TAVG'].max() - county_data['TAVG'].min()
        if temp_range <= 15:
            temp_stability_score = 100
        else:
            temp_stability_score = max(0, 100 - (temp_range - 15) * 4)
        
        # prcp score
        avg_precip = county_data['PRCP'].mean()
        if 38 <= avg_precip <= 102:
            precip_score = 100
        elif avg_precip < 38:
            precip_score = max(0, (avg_precip / 38) * 100 - 20)
        else:
            precip_score = max(0, 100 - (avg_precip - 102) * 0.6)
    

# summer comfort 
        summer_data = county_data[county_data['season'] == 'Summer']
        if not summer_data.empty:
            summer_max = summer_data['TMAX'].values[0]
            if summer_max <= 80:
                summer_comfort_score = 100
            elif summer_max <= 90:
                summer_comfort_score = max(0, 100 - (summer_max - 80) * 4)
            elif summer_max <= 100:
                summer_comfort_score = max(0, 60 - (summer_max - 90) * 5)
            else:
                summer_comfort_score = max(0, 10 - (summer_max - 100) * 2)
        else:
            summer_comfort_score = 50
        
        # winter comfort 
        winter_data = county_data[county_data['season'] == 'Winter']
        if not winter_data.empty:
            winter_min = winter_data['TMIN'].values[0]
            if winter_min >= 40:
                winter_comfort_score = 100
            elif winter_min >= 30:
                winter_comfort_score = max(0, 100 - (40 - winter_min) * 3)
            else:
                winter_comfort_score = max(0, 70 - (30 - winter_min) * 5)
        else:
            winter_comfort_score = 50
        
        # extreme scoring
        max_temp_overall = county_data['TMAX'].max()
        min_temp_overall = county_data['TMIN'].min()
        extreme_penalty = 0
        
        if max_temp_overall > 95:
            extreme_penalty += (max_temp_overall - 95) * 2
        if min_temp_overall < 30:
            extreme_penalty += (30 - min_temp_overall) * 2
        extreme_score = max(0, 100 - extreme_penalty)
        
# wind comfort score
        # all in mph
        wind_data = df_wind[df_wind['county_name'] == county]
        if not wind_data.empty:
            avg_wind = wind_data['wind_mean'].mean()
            # optimal range: 7 - 10 mph (breeze)
            if 7 <= avg_wind <= 10:
                wind_score = 100
            elif avg_wind < 7:
                # penalty for too calm 
                wind_score = max(0, 100 - (7 - avg_wind) * 10)
            else:
                # penalty for too windy
                wind_score = max(0, 100 - (avg_wind - 10) * 8)
        else:
            wind_score = 50  # Default if no wind data
        
        # total climate score : adjusted 
        climate_score = (
            temp_comfort_score * 0.18 +      # all reduced now with wind as a parameter 
            temp_stability_score * 0.13 +    
            precip_score * 0.14 +            
            summer_comfort_score * 0.28 +    
            winter_comfort_score * 0.09 +    
            extreme_score * 0.09 +           
            wind_score * 0.09                # new 9% weight
        )
        
        scores.append({
            'county_name': county,
            'climate_score': round(climate_score, 1)
        })
    
    return pd.DataFrame(scores)


scores_df = calculate_county_scores(df, df_wind) # total scores


In [11]:


#California centered map
m = folium.Map(location=[37.0, -119.5], zoom_start=6)

# CA counties GeoJSON
url = 'https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json'
response = requests.get(url)
geojson_data = response.json()

# filter for all CA counties (FIPS start with 06
ca_counties = {
    "type": "FeatureCollection",
    "features": [f for f in geojson_data['features'] if f['id'].startswith('06')]
}

fips_to_data = {} # dictionary maps FIPS to county data
for _, row in scores_df.iterrows():
    county_name = row['county_name']
    score = row['climate_score']
    
    # FIPS from original data
    county_fips = df[df['county_name'] == county_name]['county_fips'].iloc[0]
    fips_code = county_fips.replace('FIPS:', '')
    fips_to_data[fips_code] = {'name': county_name, 'score': score}

# A function for color based on score
def get_color(score):
    if score >= 95:
        return '#1a9641'  # *** Darker green
    elif score >= 90:
        return '#a6d96a'  # Lightish green
    elif score >= 80:
        return '#ffffbf'  # -- Yellow
    elif score >= 70:
        return '#fdae61'  # -- Orange
    elif score >= 60:
        return '#f46d43'  # -- Dark orange
    else:
        return '#d73027'  # Red

In [15]:
# each CA county as a separate GeoJson layer with tooltip
for feature in ca_counties['features']:
    fips = feature['id']
    if fips in fips_to_data:
        county_name = fips_to_data[fips]['name']
        score = fips_to_data[fips]['score']
        color = get_color(score)
        
        folium.GeoJson(
            feature,
            style_function=lambda x, color=color: {
                'fillColor': color,
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0.7
            },
            tooltip=f"{county_name}: {score}"
        ).add_to(m)


m.save('california_climate_map_with_wind.html')
print("Map saved") # save 

Map saved as 'california_climate_map.html'
